### Base Words (Manually Collected)

In [5]:
import gensim
import re
import numpy as np
from nltk import ngrams
import pandas as pd
import os
import sys
from pydantic import BaseModel
class Word(BaseModel):
    query: str
    id: int
    searched: bool
    scroll_ahead: int

In [2]:
arabic_news_words = list(set(["خبر", "أخبار", 
"نبأ", "أنباء",  
"حديث", "أحاديث",  # hadith (singular), ahadith (plural) - report, الحديث (al-hadith) - the news (specifically Islamic traditions)
"صحافة",  # الصحافة (as-sihaafah) - journalism, press
"إعلام",  # الإعلام (al-i'lam) - media, information
"جريدة", "جرائد",  # jaridah (singular), jaraaid (plural) - newspaper
"مجلة", "مجلات",  # majalah (singular), majallaat (plural) - magazine
"قناة", "قنوات",  # qanaah (singular), qanaat (plural) - channel (TV)
"إذاعة",  # إذاعة (idhaa'ah) - radio broadcast
"بث",  # بث (bath) - broadcast
"تقرير", "تقارير",  # taqrir (singular), taqariir (plural) - report
"خبر عاجل",  # khabar 'ajil - breaking news
"أخبار رياضية",  # akhbaar riyadiyah - sports news
"أخبار جوية",  # akhbaar jawwiyah - weather news
"سياسة",  # siyasah - politics
"اجتماع", "اجتماعات",  # ijtimaa' (singular), ijtimaaat (plural) - meeting, conference
"تصريح", "تصريحات",  # tasrih (singular), tasrihaat (plural) - statement, declaration
"تحقيق", "تحقيقات",  # tahqiq (singular), tahqiqaat (plural) - investigation, report  
"مقابلة", "مقابلات",  # muqabalah (singular), muqabalāt (plural) - interview  
"حدث", "أحداث",  # hadath (singular), aحداث (ahdaath) - event, happening
"الجزيرة",  # Al Jazeera
"العربية",  # Al Arabiya
"بي بي سي",  # BBC Arabic
"سكاي نيوز",  # Sky News Arabia
"سي إن إن",  # CNN Arabic
"قناة الشروق",  # Al-Shorouk TV
"قناة الحرة",  # Alhurra TV
"روسيا اليوم",  # RT Arabic
"فرانس 24", # France 24 (Arabic)
"DW عربية",
"أخبار", "أنباء", "حديث", "نبأ",
"الأحداث الجارية", "الشؤون الجارية",
"تقرير", "خبر", "إبلاغ",
"الصحافة", "الإعلام",
"الإعلام", "الوسائل الإعلامية",
"قناة", "محطة",
"تلفزيون",
"إذاعة",
"الجزيرة",  # Qatar
"بي بي سي",  # International
"سي إن إن",  # International
"العربية",  # UAE
"سكاي نيوز",  # UAE
"القاهرة الإخبارية",  # Egypt
"ON E",  # Egypt
"ال مبس",  # Saudi Arabia
"روتانا خليجية",
'البودكاست البحرين',
'البودكاست مصر', 
'البودكاست إيران',
'البودكاست العراق',
'البودكاست الأردن',
'البودكاست الكويت', 
'البودكاست لبنان',
'البودكاست عمان',
'البودكاست قطر', 
'البودكاست السعودية', 
'البودكاست سوريا',
'البودكاست الإمارات العربية المتحدة',
'البودكاست اليمن',
'البودكاست المصرى',
'البودكاست',
 "الجزائر", "البحرين", "مصر", "العراق", "الأردن", "الكويت", "لبنان", "ليبيا", "موريتانيا", "المغرب", 
    "عمان", "فلسطين", "قطر", "المملكة العربية السعودية", "السودان", "سوريا", "تونس", "الإمارات العربية المتحدة", "اليمن",
    "بودكاست الجزائر", "بودكاست البحرين", "بودكاست مصر", "بودكاست العراق", "بودكاست الأردن", "بودكاست الكويت", "بودكاست لبنان", 
    "بودكاست ليبيا", "بودكاست موريتانيا", "بودكاست المغرب", "بودكاست عمان", "بودكاست فلسطين", "بودكاست قطر", 
    "بودكاست المملكة العربية السعودية", "بودكاست السودان", "بودكاست سوريا", "بودكاست تونس", "بودكاست الإمارات العربية المتحدة", "بودكاست اليمن",
    "أخبار الجزائر", "أخبار البحرين", "أخبار مصر", "أخبار العراق", "أخبار الأردن", "أخبار الكويت", "أخبار لبنان", "أخبار ليبيا", 
    "أخبار موريتانيا", "أخبار المغرب", "أخبار عمان", "أخبار فلسطين", "أخبار قطر", "أخبار المملكة العربية السعودية", 
    "أخبار السودان", "أخبار سوريا", "أخبار تونس", "أخبار الإمارات العربية المتحدة", "أخبار اليمن"
]))  # Saudi Arabia]  # DW Arabic

In [3]:
df = pd.DataFrame({"query":arabic_news_words, "searched":[False for _ in arabic_news_words]})
df.head()

,query,searched
0,الأردن,False
1,أخبار فلسطين,False
2,اجتماعات,False
3,مجلات,False
4,قناة الحرة,False


In [4]:
df.to_csv(r'../data/words.csv',index=False)

In [5]:
len(df)

126

### Populate Words

In [6]:
## Preprocessing function
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

In [7]:
for arabic_word in arabic_news_words:

    t_model = gensim.models.Word2Vec.load(r'../models/full_grams_cbow_300_twitter/full_grams_cbow_300_twitter.mdl')
    token = clean_str(arabic_word).replace(" ", "_")

    if token in t_model.wv:

        most_similar = t_model.wv.most_similar( token, topn=25 )
        
        for term, score in most_similar:
            term = clean_str(term).replace(" ", "_")
            
            if term != token:
                term = term.replace("_"," ")
                df = pd.read_csv(r'../data/words.csv')
                if not (term in df['query'].values):
                    df = pd.concat([df, pd.DataFrame({"query":[term], "searched":[False]})], ignore_index=True)
                    df.to_csv(r'../data/words.csv',index=False) 
                    print(term)

سوريه
الامارات
السعوديه
المملكه
اربد
بالاردن
بيروت
الخليج
الاردنيه
بريطانيا
مادبا
سلطنه عمان
غزه
مؤتمرات
لقاءات
مشاورات
ندوات
اجتماع وزراء
زيارات
الاجتماعات
مباحثات
مؤتمر
باجتماعات
واجتماعات
الاجتماع
نشاطات
مناقشات
اجتماع مجلس
اتفاقات
اجتماعاته
المؤتمر
اجتماع طارئ
انشطه
جلسات
مداولات
اجتماعاتها
اجتماعا
ومجلات
المجلات
افلام
بروشورات
مجله
روايات
طبعات
كتب الكترونيه
مكتبات
برامج
مقالات
وبرامج
مدونات
رسومات
جرايد
اكبر موسوعه
سيديات
كتيبات
ترجمات
اغلفه
افلام ومسلسلات
اشرطه
افلام كرتون
العاب
قناه فرانس
البي بي سي
الجزيره مباشر
قناه الجزيره
قناه المنار
شاشه الجزيره
قناه الميادين
السي بي سي
قناه الاخباريه
قناه دريم
قناه الراي
قناه اون تي
قناه التحرير
قناه مكملين
العاشره مساءا
قناه نسمه
اون تي
قناه المحور
قناه العربيه
القناه المصريه
الجزيره
قناه سكاي نيوز
برنامج العاشره مساء
احدي القنوات
قناه الحدث
فوكس نيوز
وسائل اعلام تركيه
مصادر اعلاميه
فرانس
وكالات انباء
شبكه رصد
الجزيره نت
مصادر صحفيه
وسائل اعلام
وسكاي نيوز
ورويترز
نيوز»
للمغرب
المملكه المغربيه
الاردن
بالمغرب
فرنسا
الدار البيضاء
مراكش
والم

### Clean Up

In [9]:
df = pd.read_csv(r'../data/words.csv')
df.head()

,query,searched
0,الأردن,False
1,أخبار فلسطين,False
2,اجتماعات,False
3,مجلات,False
4,قناة الحرة,False


In [10]:
df['query'][417]

'لتصريحات'

In [11]:
df['query'] = df['query'].str.replace(r'[^\u0600-\u06FF\u0660-\u0669]', ' ', regex=True)

df['query'][417]

'لتصريحات'

In [12]:
df['query'] = df['query'].str.replace(r'\s+', ' ', regex=True).str.strip()
df['query'][417]

'لتصريحات'

In [13]:
df.head()

,query,searched
0,الأردن,False
1,أخبار فلسطين,False
2,اجتماعات,False
3,مجلات,False
4,قناة الحرة,False


In [15]:
len(df[df['query'] != ""])

1312

In [16]:
df = df[df['query'] != ""]
df.head()

,query,searched
0,الأردن,False
1,أخبار فلسطين,False
2,اجتماعات,False
3,مجلات,False
4,قناة الحرة,False


In [17]:
df = df.drop_duplicates()
df.duplicated().any()

False

In [18]:
len(df)

1301

In [19]:
df.to_csv(r'../data/words.csv',index=False)

In [20]:
import pandas as pd
df = pd.read_csv(r"../data/words.csv")
df.head()

,query,searched
0,الأردن,False
1,أخبار فلسطين,False
2,اجتماعات,False
3,مجلات,False
4,قناة الحرة,False


In [21]:
for row in df.values:
    print(row[0])
    break

الأردن


In [22]:
import pandas as pd
df = pd.read_csv(r"../data/words.csv")

In [23]:
df['scroll_ahead'] = 0
df['id'] = range(len(df))


In [24]:
df.head()

,query,searched,scroll_ahead,id
0,الأردن,False,0,0
1,أخبار فلسطين,False,0,1
2,اجتماعات,False,0,2
3,مجلات,False,0,3
4,قناة الحرة,False,0,4


In [25]:
df.to_csv(r'../data/words.csv',index=False)

In [6]:
import pandas as pd
df = pd.read_csv(r"../data/words.csv")

In [7]:
data_records = [ Word.model_validate(i).model_dump() for i in df.to_dict("records")]

In [8]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client['ArLip']
coll = db['words']

In [9]:
coll.insert_many(data_records)

InsertManyResult([ObjectId('6698c748f63a05d524256077'), ObjectId('6698c748f63a05d524256078'), ObjectId('6698c748f63a05d524256079'), ObjectId('6698c748f63a05d52425607a'), ObjectId('6698c748f63a05d52425607b'), ObjectId('6698c748f63a05d52425607c'), ObjectId('6698c748f63a05d52425607d'), ObjectId('6698c748f63a05d52425607e'), ObjectId('6698c748f63a05d52425607f'), ObjectId('6698c748f63a05d524256080'), ObjectId('6698c748f63a05d524256081'), ObjectId('6698c748f63a05d524256082'), ObjectId('6698c748f63a05d524256083'), ObjectId('6698c748f63a05d524256084'), ObjectId('6698c748f63a05d524256085'), ObjectId('6698c748f63a05d524256086'), ObjectId('6698c748f63a05d524256087'), ObjectId('6698c748f63a05d524256088'), ObjectId('6698c748f63a05d524256089'), ObjectId('6698c748f63a05d52425608a'), ObjectId('6698c748f63a05d52425608b'), ObjectId('6698c748f63a05d52425608c'), ObjectId('6698c748f63a05d52425608d'), ObjectId('6698c748f63a05d52425608e'), ObjectId('6698c748f63a05d52425608f'), ObjectId('6698c748f63a05d5242560